In [ ]:
! pip install -U opencv-python pillow pyyaml tqdm
import torch
from PIL import Image
model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='best_bw_400.pt')
import cv2
img = cv2.imread('Test4.jpeg')#Imagename here
img = cv2.resize(img, (960, 1240),interpolation = cv2.INTER_CUBIC)
img2=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

results = model(img2)  

a=results.xyxy[0]


for i in range(0,len(a)):
  if(a[i][5]==1):
    middle=a[i][0:5]
  if(a[i][5]==2):
    ring=a[i][0:4]

mbottom=middle[0:2]
mtop=middle[2:4]
rbottom=ring[0:2]
rtop=ring[2:4]
lengthmiddle=abs(mtop[1]-mbottom[1])
fingerwidth=max(abs(mtop[0]-mbottom[0]),abs(rtop[0]-rbottom[0]))
wrist_width=3.5*fingerwidth
watch=cv2.imread("watch2.png",-1)
watch = cv2.rotate(watch, cv2.cv2.ROTATE_90_CLOCKWISE) 
height, width = watch.shape[:2]
height_new=(wrist_width*height)/width
watch=cv2.resize(watch,(int(wrist_width),int(height_new)))
mid=(mtop[0]+mbottom[0])/2
x_off=mid-(wrist_width/2)
y_off=rbottom[1]-lengthmiddle-2*fingerwidth//1.5
x_off=int(x_off)
y_off=int(y_off-height_new)
for i in range(0,int(height_new)):
    for j in range(0,int(wrist_width)):
        if(watch[i,j][3]!=0):
          img[y_off+i,x_off+j]=watch[i,j][0:3]


img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
cv2.imwrite("Plot.jpeg",img)